# Cell counting

We will examine the use of U-Nets to perform cell counting, a crucial task to determine the number of cells in cell cultures before and after treatment.

## Cell nuclei images

We used image set BBBC039v1 Caicedo et al. 2018, available from the Broad Bioimage Benchmark Collection [Ljosa et al., Nature Methods, 2012].

This data set has a total of 200 fields of view of nuclei captured with fluorescence microscopy using the Hoechst stain. The collection has around 23,000 single nuclei manually annotated to establish a ground truth collection for segmentation evaluation.

The images are stored as TIFF files with 520x696 pixels at 16 bits. Ground truth annotations are stored as PNG files encoding masks of independent nuclei. The images have been split in three subsets: training, validation and test.

https://data.broadinstitute.org/bbbc/BBBC039/

In [3]:
from PIL import Image

# from matplotlib import pyplot as plt

# plt.figure(figsize=(15, 15))
# for i in range(4):
#     image_of_particle = Image.open("./frame_" + str(i) + ".tif")
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(image_of_particle, vmin=100, vmax=200, cmap="gray")
#     plt.axis("off")
# plt.tight_layout()
# plt.show()

## Simulations

We use ``Deeptrack2.1`` to simulate fluorescent labeled cell nuclei.

In [18]:
import deeptrack as dt
import numpy as np
from skimage import morphology

training_image_size = 128


def random_ellipse_radius():
    desired_ellipse_area = (np.random.uniform(10, 30) * dt.units.pixel) ** 2
    desired_radius_ratios = np.random.uniform(1, 4, size=2)
    desired_radius_ratios /= np.sqrt(np.prod(desired_radius_ratios))
    scale_factor = np.sqrt(desired_ellipse_area / np.pi)
    radius = desired_radius_ratios * scale_factor
    return radius


ellipse = dt.Ellipse(
    radius=random_ellipse_radius,
    intensity=lambda: np.random.uniform(0.5, 1.5),
    position=lambda: np.random.uniform(5, training_image_size - 5, size=2),
    rotation=lambda: np.random.uniform(0, 2 * np.pi),
)

optics = dt.Fluorescence(
    resolution=1e-6,
    magnification=10,
    wavelength=400e-9,
    NA=1.0,
    output_region=(0, 0, training_image_size, training_image_size),
)

synthetic_cell = (
    ellipse
    ^ (lambda: np.random.randint(2, 10))
    # >> dt.Pad(px=(10, 10, 10, 10))
    # >> dt.ElasticTransformation(alpha=50, sigma=8, order=1)
    # >> dt.CropTight()
    # >> dt.Poisson(snr=3)
)
non_overlapping_cells = dt.NonOverlapping(synthetic_cell)
image_pipeline = optics(non_overlapping_cells) >> (
    dt.Multiply(4000) >> dt.Add(100) >> np.random.poisson >> dt.Divide(2000)
)

We also need to provide target images that will be used to quantify the number of cells. As a first approximation, we simply use the segmentation map. 


In [9]:
label_pipeline = non_overlapping_cells >> dt.SampleToMasks(
    lambda: lambda x: x != 0,
    output_region=optics.output_region,
    merge_method="or",
)

In [10]:
import torch
from matplotlib import pyplot as plt

image_and_gt_pipeline = (
    image_pipeline
    & label_pipeline
    # >> dt.MoveAxis(0, 2)
    # >> dt.pytorch.ToTensor(dtype=torch.float)
)

plt.figure(figsize=(13, 9))
for i in range(3):
    image_and_gt_pipeline.update()
    image, mask = image_and_gt_pipeline()
#     plt.subplot(2, 3, i + 1)
#     plt.imshow(image, cmap="gray")
#     plt.axis("off")
#     plt.subplot(2, 3, i + 4)
#     plt.imshow(mask, cmap="gray")
#     plt.axis("off")
# plt.tight_layout()
# plt.show()

AttributeError: 'tuple' object has no attribute 'shape'

<Figure size 1300x900 with 0 Axes>

In [19]:
im = image_pipeline.update()()

AttributeError: 'tuple' object has no attribute 'size'

In [14]:
im.shape

(128, 128, 1)

In [ ]:
gt_pipeline = (
    non_overlapping_cells
    >> dt.SampleToMasks(
        lambda: lambda x: np.ones((1, 1, 1)),
        output_region=optics.output_region,
    )
    >> dt.GaussianBlur(6) * 100
)

image_and_gt_pipeline = image_pipeline & gt_pipeline

## UNet

We implement a shallow UNet since the problem does not require a large receptive field.